<a href="https://colab.research.google.com/github/kckohkoh2/fantastic-disco/blob/master/day1025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Matrix Broadcasting

In [ ]:
import torch
t0=torch.IntTensor([3]) #scalar
t1=torch.IntTensor([1,2,4]) #1D tensor
t2=torch.IntTensor([[1,2,3],[4,5,6]])
m1=t0+t1 #t0 is broadcasted to t1
m2=t1+t2 #t1 is broadcasted to t2
print(m1)
print(m2)
# PyTorch로 시작하는 딥 러닝 입문(1) - 소개|작성자 Alpha Koh

tensor([4, 5, 7], dtype=torch.int32)
tensor([[ 2,  4,  7],
        [ 5,  7, 10]], dtype=torch.int32)


 행렬곱(Matrix Multiplication)

In [ ]:
import torch
m1=torch.IntTensor([[1,2],[3,4]])
m2=torch.IntTensor([[1],[2]])
print(m1.shape)
print(m2.shape)
print(m1.matmul(m2))
print(torch.mm(m1,m2))
#PyTorch로 시작하는 딥 러닝 입문(1) - 소개|작성자 Alpha Koh

torch.Size([2, 2])
torch.Size([2, 1])
tensor([[ 5],
        [11]], dtype=torch.int32)
tensor([[ 5],
        [11]], dtype=torch.int32)


CNN으로 MNIST분류하기 

In [1]:
import torch
import torch.nn as nn

입력데이터

In [2]:
# 배치 크기 × 채널 × 높이(height) × 너비(widht)의 크기의 텐서를 선언
inputs = torch.Tensor(1, 1, 28, 28)
print('텐서의 크기 : {}'.format(inputs.shape))

텐서의 크기 : torch.Size([1, 1, 28, 28])


이제 첫번째 합성곱 층을 구현해봅시다. 1채널 짜리를 입력받아서 32채널을 뽑아내는데 커널 사이즈는 3이고 패딩은 1입니다.



In [3]:
#1채널의 입력을 받아, 32채널의 출력, 커널사이즈 3, 패딩 1
conv1 = nn.Conv2d(1, 32, 3, padding=1)
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


2번째 합성곱층 62채널의 28너비 28높이의 텐서



In [4]:
#32채널의 입력을 받아, 64채널의 출력, 커널사이즈 3, 패딩 1
conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


풀링층

In [5]:
pool = nn.MaxPool2d(2)
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


첫번 합성곱층+2x2풀링 통과후 

In [6]:
out=conv1(inputs)
print(out.shape)
out=pool(out)
print(out.shape)

torch.Size([1, 32, 28, 28])
torch.Size([1, 32, 14, 14])


2번쨰 합성곱층 통과후:  32x14x14입력데이터 64x3x3커널CNN+2x2풀링층 통과후

In [7]:
out=conv2(out)
print(out.shape)
out=pool(out)
print(out.shape)

torch.Size([1, 64, 14, 14])
torch.Size([1, 64, 7, 7])


차원 통합

In [8]:
# 첫번째 차원인 배치 차원은 그대로 두고 나머지는 펼쳐라
out = out.view(out.size(0), -1) 
print(out.shape)
print(out.size(1))

torch.Size([1, 3136])
3136


fcl(fully connected layer)통과

In [9]:
fc = nn.Linear(out.size(1),10) # input_dim = 3,136, output_dim = 10
out = fc(out)
print(out.shape)

torch.Size([1, 10])


In [10]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

클래스모델로 설계

In [11]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)
        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.fc(out)
        return out

모델정의

In [13]:
# CNN 모델 정의
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = CNN().to(device)
print(device)

cuda


비용함수와 옵티마이저 정의

In [14]:
criterion = torch.nn.CrossEntropyLoss().to(device)    # 비용 함수에 소프트맥스 함수 포함되어져 있음.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

데이터 로더

In [15]:
import torchvision.datasets as dsets
import torchvision.transforms as transforms
mnist_train = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                          train=True, # True를 지정하면 훈련 데이터로 다운로드
                          transform=transforms.ToTensor(), # 텐서로 변환
                          download=True)
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



훈련

In [16]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0
    for X, Y in data_loader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y느 ㄴ레이블.
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        avg_cost += cost / total_batch
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.238507152
[Epoch:    2] cost = 0.0610132851
[Epoch:    3] cost = 0.0428875573
[Epoch:    4] cost = 0.0353723094
[Epoch:    5] cost = 0.0283851326
[Epoch:    6] cost = 0.0235162452
[Epoch:    7] cost = 0.019279357
[Epoch:    8] cost = 0.016176898
[Epoch:    9] cost = 0.0136854239
[Epoch:   10] cost = 0.0115842912
[Epoch:   11] cost = 0.0099360114
[Epoch:   12] cost = 0.00774373394
[Epoch:   13] cost = 0.00849622861
[Epoch:   14] cost = 0.00788627751
[Epoch:   15] cost = 0.00516494177


테스트

In [18]:
# 학습을 진행하지 않을 것이므로 torch.no_grad()
mnist_test = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                          train=False, # True를 지정하면 훈련 데이터로 다운로드
                          transform=transforms.ToTensor(), # 텐서로 변환
                          download=True)
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.9893999695777893


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
